Lista de cosas por hacer:
- [ ] Más análisis
    - [x] Riqueza léxica
    - [ ] Que palabras prefieren
    - [ ] Longitud media de frases
    - [ ] Longitud media de palabras
    - [ ] De todos los textos leidos, que palabras aparecen más, cuáles menos y en que tipo son más frecuentes.
- [ ] Dividir dataset en 60/20/20 variados
- [ ] Entrenarlo y evaluar su capacidad de deteccion

Comienzo de la práctica

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd #procesado del csv pasa E/S
import matplotlib as plt #printeo
import random #operaciones matematicas
from metodos import * #importamos metodos para una mayor legibilidad

In [ ]:
import os 
for dirname, _, filenames in os.walk('/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
datos = pd.read_csv('/home/ubuntu/Escritorio/repos/segundo-cuatri/NLP/NLP-practicas_grupo/proyecto-final/archive/AI_Human.csv')
datos.head()

In [ ]:
datos_totales = (datos['generated']).count()
datos_humanos = (datos['generated'] == 0.0).sum()
print("Datos totales: ", datos_totales)
print("Humanos: ", datos_humanos)
print("IA: ", datos_totales - datos_humanos)

In [ ]:
token = nltk.word_tokenize(datos['text'][0])
nltk.pos_tag(token)

In [ ]:
tokens_ia = []
tokens_human = []
i = (len(datos['text'])//20)
while i > 0:
    valor = random.randint(0, len(datos['text']))
    token = nltk.word_tokenize(datos['text'][valor])
    if datos['generated'][i] == 1.0:
        tokens_ia.append(token)
    else:
        tokens_human.append(token)
    i -= 1


Let's find the lexical richness of the AI texts

In [ ]:
medium_richness_ia = 0
num = 0
for i in range(0, len(tokens_ia)):
    medium_richness_ia += lexical_richness(tokens_ia[i])
    num += 1
medium_richness_ia = medium_richness_ia/num
medium_richness_ia

Let's find the lexical richness of the Human texts

In [ ]:
medium_richness_human = 0
num = 0
for i in range(0, len(tokens_human)):
    medium_richness_human += lexical_richness(tokens_human[i])
    num += 1
medium_richness_human = medium_richness_human/num
medium_richness_human

We're going to chech the performance and we're gonna display the accuracy of the tagged words

In [ ]:
tagged_sents = tokens_ia

def pos_tagged_words(tagged_sents):
    tagged_words = []
    for sent in tagged_sents:
        for word in sent:
            tagged_words.append(word)
    return tagged_words

tagged_words = pos_tagged_words(tagged_sents)
tagged_words